In [1]:
import time
import os
import random
import numpy as np     
import math

import matplotlib.pyplot as plt                                   
import autograd, autograd.core, autograd.extend, autograd.tracer  
import autograd.numpy as anp      
import scipy, scipy.ndimage, scipy.sparse, scipy.sparse.linalg 

from deap import base
from deap import creator
from deap import tools

import random
import matplotlib.pyplot as plt
import seaborn as sns

import sys
import numpy as np
from mpl_toolkits.mplot3d import Axes3D


import gym
from gymnasium import spaces

from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import results_plotter

from scipy.ndimage import rotate

In [2]:
sys.path.append('../../../../../ocean_intella/ocean_intella/pipelay_vessel_design_optimisation_using_genetic_algorithms/')

In [3]:
import elitism

In [4]:
from scipy.spatial import ConvexHull

In [5]:
import networkx as nx

In [6]:
x0,y0 = 10, 10

# FEM Model

In [7]:
class ObjectView(object):
    def __init__(self, d): self.__dict__ = d
    
def get_args(normals, forces, density=1e-4):  # Manage the problem setup parameters
    width = normals.shape[0] - 1
    height = normals.shape[1] - 1
    fixdofs = np.flatnonzero(normals.ravel())
    alldofs = np.arange(2 * (width + 1) * (height + 1))
    freedofs = np.sort(list(set(alldofs) - set(fixdofs)))
    params = {
      # material properties
      'young': 1, 'young_min': 1e-9, 'poisson': 0.3, 'g': 0,
      # constraints
      'density': density, 'xmin': 0.001, 'xmax': 1.0,
      # input parameters
      'nelx': width, 'nely': height, 'mask': 1, 'penal': 3.0, 'filter_width': 1,
      'freedofs': freedofs, 'fixdofs': fixdofs, 'forces': forces.ravel(),
      # optimization parameters
      'opt_steps': 80, 'print_every': 10}
    return ObjectView(params)

def mbb_beam(width=y0, height=x0, density=1e-4, y=1, x=0, rd=-1):  # textbook beam example
    normals = np.zeros((width + 1, height + 1, 2))
    normals[0, 0, x] = 1
    normals[0, 0, y] = 1
    normals[0, -1, x] = 1
    normals[0, -1, y] = 1
    forces = np.zeros((width + 1, height + 1, 2))
    forces[-1, rd, y] = -1
    return normals, forces, density

In [8]:
def young_modulus(x, e_0, e_min, p=3):
    return e_min + x ** p * (e_0 - e_min)

def physical_density(x, args, volume_contraint=False, use_filter=True):
    x = args.mask * x.reshape(args.nely, args.nelx)  # reshape from 1D to 2D
    return gaussian_filter(x, args.filter_width) if use_filter else x  # maybe filter

def mean_density(x, args, volume_contraint=False, use_filter=True):
    return anp.mean(physical_density(x, args, volume_contraint, use_filter)) / anp.mean(args.mask)

In [9]:
def objective(x, args, volume_contraint=False, use_filter=True):
    kwargs = dict(penal=args.penal, e_min=args.young_min, e_0=args.young)
    x_phys = physical_density(x, args, volume_contraint=volume_contraint, use_filter=use_filter)
    ke     = get_stiffness_matrix(args.young, args.poisson)  # stiffness matrix
    u      = displace(x_phys, ke, args.forces, args.freedofs, args.fixdofs, **kwargs)
    c      = compliance(x_phys, u, ke, **kwargs)
    return c

In [10]:
@autograd.extend.primitive
def gaussian_filter(x, width): # 2D gaussian blur/filter
    return scipy.ndimage.gaussian_filter(x, width, mode='reflect')

def _gaussian_filter_vjp(ans, x, width): # gives the gradient of orig. function w.r.t. x
    del ans, x  # unused
    return lambda g: gaussian_filter(g, width)
autograd.extend.defvjp(gaussian_filter, _gaussian_filter_vjp)

In [11]:
def compliance(x_phys, u, ke, *, penal=3, e_min=1e-9, e_0=1):
    nely, nelx = x_phys.shape
    ely, elx = anp.meshgrid(range(nely), range(nelx))  # x, y coords for the index map

    n1 = (nely+1)*(elx+0) + (ely+0)  # nodes
    n2 = (nely+1)*(elx+1) + (ely+0)
    n3 = (nely+1)*(elx+1) + (ely+1)
    n4 = (nely+1)*(elx+0) + (ely+1)
    all_ixs = anp.array([2*n1, 2*n1+1, 2*n2, 2*n2+1, 2*n3, 2*n3+1, 2*n4, 2*n4+1])
    u_selected = u[all_ixs]  # select from u matrix

    ke_u = anp.einsum('ij,jkl->ikl', ke, u_selected)  # compute x^penal * U.T @ ke @ U
    ce = anp.einsum('ijk,ijk->jk', u_selected, ke_u)
    C = young_modulus(x_phys, e_0, e_min, p=penal) * ce.T
    return anp.sum(C)

def get_stiffness_matrix(e, nu):  # e=young's modulus, nu=poisson coefficient
    k = anp.array([1/2-nu/6, 1/8+nu/8, -1/4-nu/12, -1/8+3*nu/8,
                -1/4+nu/12, -1/8-nu/8, nu/6, 1/8-3*nu/8])
    return e/(1-nu**2)*anp.array([[k[0], k[1], k[2], k[3], k[4], k[5], k[6], k[7]],
                               [k[1], k[0], k[7], k[6], k[5], k[4], k[3], k[2]],
                               [k[2], k[7], k[0], k[5], k[6], k[3], k[4], k[1]],
                               [k[3], k[6], k[5], k[0], k[7], k[2], k[1], k[4]],
                               [k[4], k[5], k[6], k[7], k[0], k[1], k[2], k[3]],
                               [k[5], k[4], k[3], k[2], k[1], k[0], k[7], k[6]],
                               [k[6], k[3], k[4], k[1], k[2], k[7], k[0], k[5]],
                               [k[7], k[2], k[1], k[4], k[3], k[6], k[5], k[0]]])

In [12]:
def get_k(stiffness, ke):
    # Constructs sparse stiffness matrix k (used in the displace fn)
    # First, get position of the nodes of each element in the stiffness matrix
    nely, nelx = stiffness.shape
    ely, elx = anp.meshgrid(range(nely), range(nelx))  # x, y coords
    ely, elx = ely.reshape(-1, 1), elx.reshape(-1, 1)

    n1 = (nely+1)*(elx+0) + (ely+0)
    n2 = (nely+1)*(elx+1) + (ely+0)
    n3 = (nely+1)*(elx+1) + (ely+1)
    n4 = (nely+1)*(elx+0) + (ely+1)
    edof = anp.array([2*n1, 2*n1+1, 2*n2, 2*n2+1, 2*n3, 2*n3+1, 2*n4, 2*n4+1])
    edof = edof.T[0]
    x_list = anp.repeat(edof, 8)  # flat list pointer of each node in an element
    y_list = anp.tile(edof, 8).flatten()  # flat list pointer of each node in elem

    # make the global stiffness matrix K
    kd = stiffness.T.reshape(nelx*nely, 1, 1)
    value_list = (kd * anp.tile(ke, kd.shape)).flatten()
    return value_list, y_list, x_list

def displace(x_phys, ke, forces, freedofs, fixdofs, *, penal=3, e_min=1e-9, e_0=1):
    # Displaces the load x using finite element techniques (solve_coo=most of runtime)
    stiffness = young_modulus(x_phys, e_0, e_min, p=penal)
    k_entries, k_ylist, k_xlist = get_k(stiffness, ke)

    index_map, keep, indices = _get_dof_indices(freedofs, fixdofs, k_ylist, k_xlist)

    u_nonzero = solve_coo(k_entries[keep], indices, forces[freedofs], sym_pos=True)
    u_values = anp.concatenate([u_nonzero, anp.zeros(len(fixdofs))])
    return u_values[index_map]

In [13]:
def _get_dof_indices(freedofs, fixdofs, k_xlist, k_ylist):
    index_map = inverse_permutation(anp.concatenate([freedofs, fixdofs]))
    keep = anp.isin(k_xlist, freedofs) & anp.isin(k_ylist, freedofs)
    # Now we index an indexing array that is being indexed by the indices of k
    i = index_map[k_ylist][keep]
    j = index_map[k_xlist][keep]
    return index_map, keep, anp.stack([i, j])

def inverse_permutation(indices):  # reverses an index operation
    inverse_perm = np.zeros(len(indices), dtype=anp.int64)
    inverse_perm[indices] = np.arange(len(indices), dtype=anp.int64)
    return inverse_perm

In [14]:
def _get_solver(a_entries, a_indices, size, sym_pos):
    # a is (usu.) symmetric positive; could solve 2x faster w/sksparse.cholmod.cholesky(a).solve_A
    a = scipy.sparse.coo_matrix((a_entries, a_indices), shape=(size,)*2).tocsc()
    return scipy.sparse.linalg.splu(a).solve

@autograd.primitive
def solve_coo(a_entries, a_indices, b, sym_pos=False):
    solver = _get_solver(a_entries, a_indices, b.size, sym_pos)
    return solver(b)

def grad_solve_coo_entries(ans, a_entries, a_indices, b, sym_pos=False):
    def jvp(grad_ans):
        lambda_ = solve_coo(a_entries, a_indices if sym_pos else a_indices[::-1],
                            grad_ans, sym_pos)
        i, j = a_indices
        return -lambda_[i] * ans[j]
    return jvp

autograd.extend.defvjp(solve_coo, grad_solve_coo_entries,
                       lambda: print('err: gradient undefined'),
                       lambda: print('err: gradient not implemented'))

In [15]:
def fast_stopt(args, x):

    reshape = lambda x: x.reshape(args.nely, args.nelx)
    objective_fn = lambda x: objective(reshape(x), args)
#     constraint = lambda params: mean_density(reshape(params), args) - args.density
    constraint = lambda params: mean_density(reshape(params), args) 
    value = objective_fn(x)
    const = constraint(x)
    return value, const

In [16]:
def draw(X):  
    plt.figure(dpi=50) 
    print('\nFinal Cantilever beam design:')
    plt.imshow(X) 
    plt.show()

# Genetic Modelling

In [17]:
# Genetic Algorithm constants:
POPULATION_SIZE = 300
P_CROSSOVER = 0.9  # probability for crossover
P_MUTATION = 0.1   # probability for mutating an individual
MAX_GENERATIONS = 6000
HALL_OF_FAME_SIZE = 30

In [18]:
PENALTY_VALUE = 1000.0    

In [19]:
# set the random seed:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [20]:
toolbox = base.Toolbox()

In [21]:
# define a single objective, minimizing fitness strategy:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

In [22]:
# create the Individual class based on list:
creator.create("Individual", list, fitness=creator.FitnessMin)

In [23]:
# create an operator that randomly returns 0 or 1:
toolbox.register("zeroOrOne", random.randint, 0, 1)

In [24]:
# create the individual operator to fill up an Individual instance:
toolbox.register("individualCreator", 
                 tools.initRepeat,
                 creator.Individual,
                 toolbox.zeroOrOne, 
                 x0*y0+4)

In [25]:
# create the population operator to generate a list of individuals:
toolbox.register("populationCreator", 
                 tools.initRepeat, 
                 list, 
                 toolbox.individualCreator)

In [26]:
# convert a binary list to decimal
def binatodeci(binary):
    return sum(val*(2**idx) for idx, val in enumerate(reversed(binary)))

In [27]:
# fitness calculation
def staticFEM(individual):
    
    args = get_args(*mbb_beam(rd=-1))
    
    x = np.array([t if t==1 else 1e-4 for t in individual[:-4]]) 
    
    try:
        tmp, const = fast_stopt(args, x)
        if const>(55+binatodeci(individual[-4:]))/100:
            return PENALTY_VALUE, 
    except:
        return PENALTY_VALUE, 
    
    
    return tmp,  # return a tuple

In [28]:
toolbox.register("evaluate", staticFEM)

# genetic operators:
toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=1.0/(x0*y0+4))

In [29]:
# Genetic Algorithm flow:
def main():

    # create initial population (generation 0):
    population = toolbox.populationCreator(n=POPULATION_SIZE)

    # prepare the statistics object:
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.mean)

    # define the hall-of-fame object:
    hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

    # perform the Genetic Algorithm flow with hof feature added:
    population, logbook = elitism.eaSimpleWithElitism(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                              ngen=MAX_GENERATIONS, stats=stats, halloffame=hof, verbose=True)

    # print best solution found:
    best = hof.items[0]
    print("-- Best Individual = ", best)
    print("-- Best Fitness = ", best.fitness.values[0])
    print()
    
    x = np.array(best[:-4]).astype(np.float64).reshape(x0,y0)
    
    args = get_args(*mbb_beam(rd=-1))
    print(fast_stopt(args, x))
    
    draw(x)
    

    # extract statistics:
    minFitnessValues, meanFitnessValues = logbook.select("min", "avg")

    # plot statistics:
    sns.set_style("whitegrid")
    plt.plot(minFitnessValues, color='red')
    plt.plot(meanFitnessValues, color='green')
  
    plt.xlabel('Generation')
    plt.ylabel('Min / Average Fitness')
    plt.title('Min and Average fitness over Generations')
    plt.show()    
    return x

In [ ]:
start = time.time()
top_=main()
end = time.time() 

gen	nevals	min   	avg        
0  	300   	40.597	1.06682e+06
1  	247   	40.597	700.548    
2  	252   	40.597	615.067    
3  	253   	40.597	142.701    
4  	255   	39.3084	164.824    
5  	248   	35.7276	147.409    
6  	246   	35.7276	109.762    
7  	235   	35.7276	118.618    
8  	248   	34.113 	134.151    
9  	241   	31.655 	118.004    
10 	241   	31.2711	153.032    
11 	242   	29.4082	144.237    
12 	249   	27.0919	120.089    
13 	246   	25.8896	102.272    
14 	226   	25.8896	106.919    
15 	251   	25.8896	105.909    
16 	250   	25.3352	143.813    
17 	254   	24.4339	133.48     
18 	244   	23.4157	132.494    
19 	248   	23.4157	170.547    
20 	241   	23.4157	160.353    
21 	230   	22.9005	127.399    
22 	239   	22.8485	172.15     
23 	258   	22.7728	174.982    
24 	236   	22.6288	141.892    
25 	250   	22.3473	183.787    
26 	248   	22.3369	183.493    
27 	254   	21.6602	202.927    
28 	248   	21.5688	205.944    
29 	229   	21.5688	143.855    
30 	238   	21.3615	182.639    
31 	244   	21

264	242   	19.2874	35.6784    
265	251   	19.2874	39.074     
266	240   	19.2874	32.442     
267	248   	19.2874	39.041     
268	247   	19.2874	45.5068    
269	242   	19.2874	55.3366    
270	245   	19.2874	38.9584    
271	258   	19.2874	35.8118    
272	252   	19.2874	42.2612    
273	246   	19.2874	42.2194    
274	245   	19.2874	35.6899    
275	256   	19.2874	55.3159    
276	243   	19.2874	45.4903    
277	249   	19.2874	35.6999    
278	251   	19.2874	45.4995    
279	250   	19.2874	42.2226    
280	250   	19.2874	42.2865    
281	245   	19.2874	45.531     
282	250   	19.2874	29.1999    
283	248   	19.2874	32.4769    
284	242   	19.2874	38.9269    
285	242   	19.2874	48.7638    
286	254   	19.2874	45.5988    
287	240   	19.2874	52.0831    
288	243   	19.2874	29.216     
289	241   	19.2874	45.485     
290	247   	19.2874	61.8858    
291	232   	19.2874	42.288     
292	239   	19.2874	35.6877    
293	249   	19.2874	39.0666    
294	256   	19.2874	58.6116    
295	254   	19.2874	42.2645    
296	240 

529	243   	19.2725	35.6755    
530	254   	19.2725	38.9728    
531	253   	19.2725	38.9934    
532	247   	19.2725	25.9073    
533	252   	19.2725	38.9445    
534	245   	19.2725	42.2527    
535	238   	19.2725	74.9912    
536	258   	19.2725	52.0711    
537	240   	19.2725	42.2475    
538	232   	19.2725	39.0362    
539	246   	19.2725	35.6635    
540	249   	19.2725	29.158     
541	249   	19.2725	42.2685    
542	235   	19.2725	48.7488    
543	237   	19.2725	48.8037    
544	243   	19.2725	52.0197    
545	247   	19.2725	42.2574    
546	247   	19.2725	52.0335    
547	254   	19.2725	45.6384    
548	243   	19.2725	42.2478    
549	229   	19.2725	38.993     
550	254   	19.2725	48.7621    
551	241   	19.2725	61.8853    
552	255   	19.2725	42.2854    
553	255   	19.2725	45.5005    
554	251   	19.2725	35.685     
555	242   	19.2725	38.9748    
556	254   	19.2725	38.9641    
557	240   	19.2725	45.5054    
558	236   	19.2725	22.7249    
559	250   	19.2725	29.1514    
560	250   	19.2725	32.4193    
561	254 

794	247   	19.2725	71.675     
795	239   	19.2725	45.5279    
796	252   	19.2725	39.0135    
797	240   	19.2725	45.4618    
798	245   	19.2725	29.1916    
799	236   	19.2725	38.982     
800	253   	19.2725	25.945     
801	251   	19.2725	39.0032    
802	254   	19.2725	45.4987    
803	253   	19.2725	42.2025    
804	242   	19.2725	55.3419    
805	247   	19.2725	48.7424    
806	246   	19.2725	42.2295    
807	256   	19.2725	48.8056    
808	244   	19.2725	25.8936    
809	249   	19.2725	39.0177    
810	243   	19.2725	32.4403    
811	252   	19.2725	32.3989    
812	243   	19.2725	55.27      
813	234   	19.2725	35.7312    
814	247   	19.2725	32.4478    
815	250   	19.2725	38.9896    
816	249   	19.2725	32.4905    
817	242   	19.2725	52.0474    
818	238   	19.2725	29.2262    
819	245   	19.2725	39.0132    
820	239   	19.2725	58.5835    
821	243   	19.2725	48.8249    
822	243   	19.2725	48.7741    
823	242   	19.2725	45.4886    
824	243   	19.2725	35.6803    
825	248   	19.2725	45.5211    
826	240 

1057	248   	19.2725	42.2764    
1058	237   	19.2725	42.2328    
1059	246   	19.2725	45.4719    
1060	252   	19.2725	52.0446    
1061	251   	19.2725	38.915     
1062	240   	19.2725	32.4662    
1063	249   	19.2725	45.5548    
1064	236   	19.2725	35.7354    
1065	243   	19.2725	45.5949    
1066	255   	19.2725	55.3179    
1067	247   	19.2725	52.049     
1068	244   	19.2725	42.2311    
1069	245   	19.2725	35.699     
1070	245   	19.2725	35.6568    
1071	245   	19.2725	58.5735    
1072	239   	19.2725	42.2509    
1073	246   	19.2725	61.8299    
1074	246   	19.2725	48.8116    
1075	243   	19.2725	58.6697    
1076	228   	19.2725	32.4352    
1077	244   	19.2725	52.0629    
1078	238   	19.2725	42.2701    
1079	250   	19.2725	52.0847    
1080	240   	19.2725	35.7313    
1081	254   	19.2725	32.4034    
1082	247   	19.2725	45.5145    
1083	260   	19.2725	35.7566    
1084	256   	19.2725	32.3969    
1085	248   	19.2725	42.3621    
1086	242   	19.2725	48.8117    
1087	250   	19.2725	39.0335    
1088	242

...

2085	249   	19.2725	35.655     
2086	247   	19.2725	42.2195    
2087	238   	19.2725	61.8176    
2088	227   	19.2725	42.2707    
2089	256   	19.2725	45.5       
2090	239   	19.2725	45.5255    
2091	240   	19.2725	42.2372    
2092	245   	19.2725	55.3126    
2093	247   	19.2725	48.8336    
2094	240   	19.2725	32.4786    
2095	248   	19.2725	42.2303    
2096	256   	19.2725	45.5497    
2097	254   	19.2725	42.3094    
2098	259   	19.2725	35.6687    
2099	249   	19.2725	42.2369    
2100	236   	19.2725	42.238     
2101	247   	19.2725	48.7794    
2102	240   	19.2725	38.9246    
2103	256   	19.2725	32.4376    
2104	252   	19.2725	25.9171    
2105	243   	19.2725	48.8199    
2106	252   	19.2725	38.9868    
2107	243   	19.2725	29.1177    
2108	247   	19.2725	25.8605    
2109	247   	19.2725	48.8025    
2110	246   	19.2725	42.1911    
2111	239   	19.2725	29.1555    
2112	256   	19.2725	32.4626    
2113	243   	19.2725	48.7861    
2114	231   	19.2725	42.3199    
2115	246   	19.2725	32.4238    
2116	246

2342	249   	19.2725	45.5359    
2343	253   	19.2725	58.5562    
2344	247   	19.2725	32.4199    
2345	236   	19.2725	32.4773    
2346	238   	19.2725	25.8739    
2347	242   	19.2725	42.2022    
2348	243   	19.2725	22.5955    
2349	253   	19.2725	25.937     
2350	254   	19.2725	35.6743    
2351	257   	19.2725	29.1796    
2352	238   	19.2725	35.7458    
2353	251   	19.2725	35.7329    
2354	253   	19.2725	32.4939    
2355	249   	19.2725	42.2843    
2356	241   	19.2725	42.3118    
2357	252   	19.2725	35.8132    
2358	252   	19.2725	32.4108    
2359	244   	19.2725	35.6949    
2360	245   	19.2725	38.9717    
2361	254   	19.2725	45.4872    
2362	252   	19.2725	45.4866    
2363	249   	19.2725	35.692     
2364	254   	19.2725	35.7129    
2365	251   	19.2725	22.6001    
2366	252   	19.2725	39.0169    
2367	230   	19.2725	38.9511    
2368	245   	19.2725	35.6825    
2369	246   	19.2725	25.9041    
2370	238   	19.2725	38.9267    
2371	251   	19.2725	35.6707    
2372	249   	19.2725	19.3699    
2373	248

2599	245   	19.2725	52.0402    
2600	237   	19.2725	52.0101    
2601	248   	19.2725	32.4258    
2602	253   	19.2725	32.474     
2603	249   	19.2725	38.9822    
2604	252   	19.2725	38.9246    
2605	253   	19.2725	48.7774    
2606	232   	19.2725	42.2304    
2607	239   	19.2725	42.2916    
2608	244   	19.2725	48.7602    
2609	249   	19.2725	42.2144    
2610	240   	19.2725	45.4877    
2611	256   	19.2725	35.6616    
2612	239   	19.2725	52.0671    
2613	242   	19.2725	38.9639    
2614	237   	19.2725	35.7343    
2615	234   	19.2725	32.414     
2616	243   	19.2725	42.2226    
2617	251   	19.2725	42.2437    
2618	254   	19.2725	55.2776    
2619	244   	19.2725	42.2392    
2620	247   	19.2725	42.2332    
2621	240   	19.2725	42.2631    
2622	248   	19.2725	38.9801    
2623	243   	19.2725	42.2362    
2624	251   	19.2725	48.8813    
2625	240   	19.2725	38.9504    
2626	255   	19.2725	45.4706    
2627	241   	19.2725	38.9521    
2628	240   	19.2725	45.4756    
2629	254   	19.2725	48.7958    
2630	245

2856	240   	19.2725	42.2678    
2857	250   	19.2725	35.6784    
2858	239   	19.2725	38.9789    
2859	240   	19.2725	45.5815    
2860	248   	19.2725	45.5124    
2861	245   	19.2725	29.1391    
2862	237   	19.2725	52.0168    
2863	247   	19.2725	39.0268    
2864	250   	19.2725	42.2223    
2865	257   	19.2725	38.9781    
2866	244   	19.2725	32.4421    
2867	244   	19.2725	58.5951    
2868	246   	19.2725	45.5301    
2869	244   	19.2725	39.027     
2870	240   	19.2725	25.913     
2871	250   	19.2725	42.2359    
2872	242   	19.2725	25.894     
2873	250   	19.2725	32.4414    
2874	237   	19.2725	32.4642    
2875	249   	19.2725	35.7244    
2876	257   	19.2725	48.7776    
2877	243   	19.2725	32.5062    
2878	259   	19.2725	35.6737    
2879	250   	19.2725	32.4384    
2880	232   	19.2725	38.9664    
2881	258   	19.2725	45.5211    
2882	243   	19.2725	29.2635    
2883	239   	19.2725	38.9894    
2884	245   	19.2725	58.5995    
2885	252   	19.2725	52.0263    
2886	239   	19.2725	35.6772    
2887	257

3113	236   	19.2725	42.2074    
3114	235   	19.2725	48.8451    
3115	240   	19.2725	35.6508    
3116	244   	19.2725	42.2406    
3117	248   	19.2725	52.0424    
3118	241   	19.2725	45.5205    
3119	248   	19.2725	35.7431    
3120	240   	19.2725	48.7629    
3121	253   	19.2725	52.1064    
3122	245   	19.2725	32.4827    
3123	257   	19.2725	55.3073    
3124	248   	19.2725	32.4035    
3125	252   	19.2725	32.3766    
3126	237   	19.2725	42.1762    
3127	244   	19.2725	29.1815    
3128	247   	19.2725	32.4336    
3129	244   	19.2725	32.4562    
3130	257   	19.2725	29.1851    
3131	236   	19.2725	42.2318    
3132	239   	19.2725	35.6984    
3133	247   	19.2725	48.7573    
3134	253   	19.2725	38.9702    
3135	250   	19.2725	29.161     
3136	244   	19.2725	32.4543    
3137	234   	19.2725	22.6812    
3138	227   	19.2725	29.1453    
3139	244   	19.2725	32.4067    
3140	246   	19.2725	42.2332    
3141	251   	19.2725	38.9543    
3142	240   	19.2725	48.8333    
3143	257   	19.2725	38.972     
3144	242

In [ ]:
print('Total time taken: {} min'.format((end - start)/60))

In [ ]:
35*150/0.8882

In [ ]:
.

In [ ]:
class Model:
    def __init__(self, x):
        self.flag_ = True
#         self.flag_ = False
        self.n, self.m = x.shape
        self.actions_dic={} 
    
        k=0
        for i in range(self.n):
            for j in range(self.m):
                count = 0
                if x[i][j]==1:
                    
                
                    if 0<=i+1<self.n and x[i+1][j]==1:
                        count+=1
                    if 0<=i-1<self.n and x[i-1][j]==1:
                        count+=1 
                    if 0<=j+1<self.m and x[i][j+1]==1:
                        count+=1
                    if 0<=j-1<self.m and x[i][j-1]==1:
                        count+=1   
#                     if i==0 or i== self.n-1 or j==0 or j==self.m-1:
#                         count+=1
                
                
                    if count<4:
                        self.actions_dic[k]=(i,j)
                        k+=1
                        
                elif  x[i][j]==0:
                    x[i][j]+=1e-4
        
    def action_space_(self, action, X):
        x,y=self.actions_dic[action]
        X[x][y]=1e-4
        
    def draw(self,X):  
        plt.figure(dpi=50) 
        print('\nFinal Cantilever beam design:')
        plt.imshow(X) 
        plt.show()

In [ ]:
class CantileverEnv(gym.Env):
    
    metadata = {"render.modes": ["human"]}

    def __init__(self,topology, normalized_image: bool = False):
        super().__init__()
        
        
        self.rd=0
        self.args = get_args(*mbb_beam(rd=self.rd))
        
#         DIM=self.args.nelx*self.args.nely

        self.topology=topology.copy()
#         mp, const = fast_stopt(self.args, topology)
        self.x = self.topology.copy()
#         mp, const = fast_stopt(self.args, self.x)
#         print(mp)
        self.M=Model(self.x)
#         print(len(self.M.actions_dic))

        N_DISCRETE_ACTIONS=len(self.M.actions_dic)
        
        self.action_space = spaces.Discrete(N_DISCRETE_ACTIONS)
        self.observation_space = spaces.Box(low=0,
                                            high=255,
                                            shape=(self.args.nely, self.args.nelx, 1),
                                            dtype=np.float64)
        
        
        
        self.reward=0
        self.step_=0
        self.needs_reset = True
        
    def step(self, action):
        
        self.args = get_args(*mbb_beam(rd=self.rd))
#         print(self.x)
        mp, const = fast_stopt(self.args, self.x)
#         print(mp)
        self.M.action_space_(action, self.x)
#         print(self.x)
        tmp, const = fast_stopt(self.args, self.x)
#         print()
#         print(action)
        self.step_+=1
        
        self.reward+=(1/tmp)**2
       
        done=False
                  
        if self.step_>self.M.n*self.M.m:
            done=True
            
        if const>0.7:
#             self.reward-=1
            done=True
            
        if self.needs_reset:
            raise RuntimeError("Tried to step environment that needs reset")
            
        if done:
            self.needs_reset = True
                         
                
#         resized_x = resize(self.x, ( self.args.nely, self.args.nelx), anti_aliasing=True)  
#         resized_x = (resized_x * 255).astype(np.uint8).reshape(x0,y0,1)

        return self.x.reshape(x0,y0,1), self.reward, done, False, {}

    def reset(self, seed=0):
        
        if not self.M.flag_:
            self.rd=random.choice([0,2,-2])
        else:
            self.rd=-1
           
        self.x = self.topology.copy()

        self.reward=0
        self.needs_reset = False
        self.step_=0
        
#         resized_x = resize(self.x, (self.args.nely, self.args.nelx), anti_aliasing=True)
#         resized_x = (resized_x * 255).astype(np.uint8).reshape(x0,y0,1)

        return self.x.reshape(x0,y0,1), {}

    def render(self, mode="human"):
        self.M.draw(self.x)    

    def close(self):
        pass

In [ ]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq: int, log_dir: str, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, "best_model")
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

            # Retrieve training reward
            x, y = ts2xy(load_results(self.log_dir), "timesteps")
            if len(x) > 0:
                # Mean training reward over the last 100 episodes
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print(f"Num timesteps: {self.num_timesteps}")
                    print(
                        f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}"
                    )

                # New best model, you could save the agent here
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    # Example for saving best model
                    if self.verbose > 0:
                        print(f"Saving new best model to {self.save_path}.zip")
                    self.model.save(self.save_path)

        return True

In [ ]:
# ts=5e6
ts=0.5e6

In [ ]:
# Create log dir
log_dir = "/tmp/gym_gen_1b/"
os.makedirs(log_dir, exist_ok=True)

# Create and wrap the environment
env = CantileverEnv(topology=top_)
# Logs will be saved in log_dir/monitor.csv
env = Monitor(env, log_dir)
check_env(env)

In [ ]:
# callback = SaveOnBestTrainingRewardCallback(check_freq=5000, log_dir=log_dir)
callback = SaveOnBestTrainingRewardCallback(check_freq = 10000, log_dir = log_dir)

In [ ]:
start = time.time()
model = PPO("MlpPolicy", env).learn(total_timesteps=ts, callback=callback)
# model = PPO.load(log_dir + "best_model.zip", env=env).learn(total_timesteps=ts, callback=callback)
end = time.time()   

In [ ]:
print('Total time taken: {} min'.format((end - start)/60))

### Inference

In [ ]:
# env.M.flag_=True
env.M.flag_=False
obs=env.reset()
obs=obs[0]

In [ ]:
i=0
while i<2000:
    action, _states = model.predict(obs)
    obs, rewards, dones, _ , info = env.step(int(action))
    if dones: 
        break
    i+=1

In [ ]:
print(i)

In [ ]:
fast_stopt(env.args, env.x)

In [ ]:
env.render()

In [ ]:
results_plotter.plot_results([log_dir], ts, results_plotter.X_TIMESTEPS, "CantileverEnv")

In [ ]:
obs=env.reset()
obs=obs[0]

In [ ]:
# Load the agent
model_best = PPO.load(log_dir + "best_model.zip", env=env)

In [ ]:
i=0
while i<2000:
    action, _states = model_best.predict(obs)
    
    obs, rewards, dones, _ , info = env.step(int(action))
    if dones:
        break
    i+=1

In [ ]:
i

In [ ]:
env.render()

In [ ]:
# https://stable-baselines3.readthedocs.io/en/master/guide/save_format.html

In [ ]:
fast_stopt(env.args, env.x)